In [ ]:
!wget https://downloads.dbpedia.org/repo/dbpedia/generic/anchor-text/2022.12.01/anchor-text_lang=en.ttl.bz2
!bunzip2 anchor-text_lang=en.ttl.bz2

In [ ]:
from redis import StrictRedis

redis_forms = StrictRedis(host='localhost', port=7979, db=0)

In [ ]:
NS_RESOURCE = 'http://dbpedia.org/resource/'
NS_RESOURCE_LEN = len(NS_RESOURCE)

In [ ]:
import sys
from rdflib import Graph, URIRef, BNode, Literal
from rdflib.plugins.parsers import ntriples
from rdflib.exceptions import ParserError as ParseError

class CustomSink(ntriples.NTGraphSink):
    def __init__(self, graph):
        super().__init__(graph)
        self.counter = 0
    def triple(self, s, p, o):
        entity = str(s)[NS_RESOURCE_LEN:]
        redis_forms.hincrby(o, entity, 1)
        self.counter += 1
        if self.counter % 1_000 == 0:
            print(f"{round(self.counter / 1_000, 0)} k")

class CustomParser(ntriples.W3CNTriplesParser):
    def parse(self, f):
        if not hasattr(f, "read"):
            raise ParseError("Item to parse must be a file-like object.")

        if not hasattr(f, "encoding") and not hasattr(f, "charbuffer"):
            # someone still using a bytestream here?
            f = codecs.getreader("utf-8")(f)

        self.file = f  # type: ignore[assignment]
        self.buffer = ""
        while True:
            self.line = self.readline()
            if self.line is None:
                break
            try:
                self.parseline()
            except ParseError:
                pass # ignore parse errors
        return self.sink

fname = r"anchor-text_lang=en.ttl"
sink1 = CustomSink(Graph())
p1 = CustomParser(sink1)
with open(fname, encoding='utf-8') as f:
    p1.parse(f)

In [ ]:
redis_forms.save()